In [ ]:
import sys
import numpy as np
import sacc
import yaml
import pandas as pd
import firecrown

import firecrown.likelihood.weak_lensing as wl
import firecrown.likelihood.number_counts as nc
from firecrown.likelihood.two_point import TwoPoint
from firecrown.likelihood.gaussian import ConstGaussian

In [ ]:
from firecrown.metadata.two_point import extract_all_data_types_xi_thetas
from firecrown.metadata.two_point import extract_all_data_xi_thetas

s = sacc.Sacc.load_fits("examples/des_y1_3x2pt/des_y1_3x2pt_sacc_data.fits")
two_point_xi_thetas = extract_all_data_xi_thetas(s)

In [ ]:
import pacc
cle = pacc.ClsEnsemble(s, label="DES Y1 real space")
pacc.plot_cls([cle], cle.pairs)

In [ ]:
from firecrown.likelihood.two_point import TwoPoint
from firecrown.likelihood.weak_lensing import WeakLensingFactory
from firecrown.likelihood.number_counts import NumberCountsFactory
from firecrown.utils import base_model_from_yaml

# WeakLensing systematics -- global
ia_systematic = wl.LinearAlignmentSystematicFactory()
# WeakLensing systematics -- per-bin
wl_photoz = wl.PhotoZShiftFactory()
wl_mult_bias = wl.MultiplicativeShearBiasFactory()

# NumberCounts systematics -- global
# As for Firecrown 1.8.0a0, we do not have any global systematics for number counts
# NumberCounts systematics -- per-bin
nc_photoz = nc.PhotoZShiftFactory()

wlf = wl.WeakLensingFactory(
    per_bin_systematics=[wl_mult_bias, wl_photoz],
    global_systematics=[ia_systematic],
)
ncf = nc.NumberCountsFactory(
    per_bin_systematics=[nc_photoz],
    global_systematics=[],
)

In [ ]:
two_point_list = TwoPoint.from_metadata_real(
    two_point_xi_thetas,
    wl_factory=wlf, 
    nc_factory=ncf,
    check_consistence=True)

In [ ]:
import pyccl
from firecrown.modeling_tools import ModelingTools
from firecrown.parameters import ParamsMap

req_params = two_point_list.required_parameters()
default_values = req_params.get_default_values()
params = ParamsMap(default_values)

ccl_cosmo = pyccl.CosmologyVanillaLCDM()
ccl_cosmo.compute_nonlin_power()
tools = ModelingTools()
tools.update(params)
tools.prepare(ccl_cosmo)

two_point_list.update(params)

theory= []
for tp in two_point_list:
    t = tp.compute_theory_vector(tools)
    theory.append(np.array(t))
theory = np.concatenate(theory)

In [ ]:
t_cle = pacc.ClTheoryEnsemble(cle, theory, label="DES Y1 real space")
pacc.plot_cls([cle, t_cle], cle.pairs)